Workspace Information:

Workspace Namespace
aou-rw-37d4bab0

Dataset
All of Us Controlled Tier Dataset v8

Creation Date
Wed Feb 05 2025

Last Updated
Wed Feb 05 2025

In [ ]:
import pandas
import os

dataset_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_start_datetime,
        c_type.concept_name as condition_type_concept_name,
        visit.concept_name as visit_occurrence_concept_name,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (702953)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (702953) 
                            AND is_standard = 0 )) criteria ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_condition_df = pandas.read_gbq(
    dataset_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

In [ ]:
import numpy as np
hospitalization_indicators = [
    'Inpatient Visit', 
    'Inpatient Hospital', 
    'Emergency Room and Inpatient Visit'
]

valid_indicators = ['Outpatient Visit', 'Telehealth', 'Emergency Room Visit',
       'Office Visit', 'Emergency Room and Inpatient Visit', 'Inpatient Visit',
       'Laboratory Visit', 'Non-hospital institution Visit',
       'Ambulatory Radiology Clinic / Center', 'Observation Room',
       'Ambulatory Infusion Therapy Clinic / Center',
       'Ambulatory Rehabilitation Visit', 'Home Visit', 'Inpatient Hospital',
       'Pharmacy', 'Outpatient Hospital']

# Create a new column 'inpatient' where the condition is met
dataset_condition_df = dataset_condition_df[dataset_condition_df['visit_occurrence_concept_name'].isin(valid_indicators)]
dataset_condition_df['inpatient'] = np.where(
    dataset_condition_df['visit_occurrence_concept_name'].isin(hospitalization_indicators),
    1,  # Value if condition is True
    0   # Value if condition is False
)
dataset_condition_df = dataset_condition_df.drop(columns = ['visit_occurrence_concept_name'])

In [ ]:
condition_status_mapping = {
    "None": np.nan,
    "No matching concept": np.nan,
    "Final diagnosis (discharge)": "Inpatient",
    "Primary diagnosis": "Primary",
    "Emergency use of U07.1 | COVID-19": np.nan,
    "Secondary diagnosis": "Secondary",
    "Suggested billing diagnosis": "Billing",
    "Primary discharge diagnosis": "Primary",
    "Admission diagnosis": "Inpatient",
    "Preliminary diagnosis": "Primary",
    "Discharge diagnosis": "Inpatient",
    "Secondary discharge diagnosis": "Secondary",
    "Confirmed diagnosis": "Primary",
    "Established diagnosis": "Primary",
    "Problem resolved": np.nan,
}
dataset_condition_df["condition_status_concept_name"] = dataset_condition_df["condition_status_concept_name"].map(condition_status_mapping)
# Define mapping
mapping = {
    "Primary Condition": "Primary",
    "Primary diagnosis": "Primary",
    "Secondary Condition": "Secondary",
    "Secondary diagnosis": "Secondary",
    "EHR billing record": "EHR",
    "EHR encounter record": "EHR",
    "EHR": "EHR",
    "EHR billing diagnosis": "EHR",
    "EHR encounter diagnosis": "EHR",
    "EHR problem list": "EHR",
    "EHR problem list entry": "EHR",
    "EHR discharge record": "EHR",
    "EHR Chief Complaint": "EHR",
    "Observation recorded from EHR": "EHR",
    "Outpatient header - 1st position": "Outpatient",
    "Outpatient header - 2nd position": "Outpatient",
    "Claim": "Claims",
    "Inpatient claim": "Inpatient",
    "Inpatient header - 1st position": "Inpatient",
    "Inpatient header - 2nd position": "Inpatient",
    "Inpatient header - 3rd position": "Inpatient",
    "Inpatient header - 4th position": "Inpatient",
    "Inpatient header - 5th position": "Inpatient",
    "Inpatient header - 6th position": "Inpatient",
    "Inpatient header - 8th position": "Inpatient",
    "Inpatient header - 11th position": "Inpatient",
    "Inpatient header - 13th position": "Inpatient",
    "Inpatient header - 15th position": "Inpatient",
    "Admission diagnosis": "Inpatient",
    "Discharge diagnosis": "Inpatient",
    "Condition tested for by diagnostic procedure": "Secondary"
}
dataset_condition_df["condition_type_concept_name"] = dataset_condition_df["condition_type_concept_name"].map(mapping)
dataset_condition_df['EHR'] = np.where((dataset_condition_df['condition_type_concept_name'] == 'EHR'), 1, 0)
dataset_condition_df['Claims'] = np.where((dataset_condition_df['condition_type_concept_name'] == 'Claims') | (dataset_condition_df['condition_status_concept_name'] == 'Billing'), 1, 0)
dataset_condition_df['Primary'] = np.where((dataset_condition_df['condition_type_concept_name'] == 'Primary') | (dataset_condition_df['condition_status_concept_name'] == 'Primary'), 1, 0)
dataset_condition_df['Secondary'] = np.where((dataset_condition_df['condition_type_concept_name'] == 'Secondary') | (dataset_condition_df['condition_status_concept_name'] == 'Secondary'), 1, 0)
dataset_condition_df[dataset_condition_df['condition_type_concept_name'] == 'Inpatient']['inpatient'] = 1
dataset_condition_df[dataset_condition_df['condition_status_concept_name'] == 'Inpatient']['inpatient'] = 1

In [ ]:
import pandas
import os

dataset_observation_sql = """
    SELECT
        observation.person_id,
        observation.observation_datetime,
        o_type.concept_name as observation_type_concept_name,
        o_qualifier.concept_name as qualifier_concept_name,
        o_visit.concept_name as visit_occurrence_concept_name
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
        WHERE
            (
                observation_source_concept_id IN (702953)
            )  
            AND (
                observation.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (702953) 
                            AND is_standard = 0 )) criteria ) )
            )) observation 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_standard_concept 
            ON observation.observation_concept_id = o_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_type 
            ON observation.observation_type_concept_id = o_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_qualifier 
            ON observation.qualifier_concept_id = o_qualifier.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON observation.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_visit 
            ON v.visit_concept_id = o_visit.concept_id"""

dataset_observation_df = pandas.read_gbq(
    dataset_observation_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

In [ ]:
import numpy as np
hospitalization_indicators = [
    'Inpatient Visit', 
    'Inpatient Hospital', 
    'Emergency Room and Inpatient Visit'
]

# Create a new column 'inpatient' where the condition is met
dataset_observation_df['inpatient'] = np.where(
    dataset_observation_df['visit_occurrence_concept_name'].isin(hospitalization_indicators),
    1,  # Value if condition is True
    0   # Value if condition is False
)
dataset_observation_df = dataset_observation_df.drop(columns = ['visit_occurrence_concept_name'])

In [ ]:
condition_status_mapping = {
    "Principal diagnosis": "Primary",
    "Clinical diagnosis": "Primary",
    "Secondary diagnosis": "Secondary",
    "Suggested billing diagnosis": "Billing"
}
dataset_observation_df["qualifier_concept_name"] = dataset_observation_df["qualifier_concept_name"].map(condition_status_mapping)
# Define mapping
mapping = {
    "Primary Condition": "Primary",
    "Primary diagnosis": "Primary",
    "Secondary Condition": "Secondary",
    "Secondary diagnosis": "Secondary",
    "EHR billing record": "EHR",
    "EHR encounter record": "EHR",
    "EHR": "EHR",
    "EHR billing diagnosis": "EHR",
    "EHR encounter diagnosis": "EHR",
    "EHR problem list": "EHR",
    "EHR problem list entry": "EHR",
    "EHR discharge record": "EHR",
    "EHR Chief Complaint": "EHR",
    "Observation recorded from EHR": "EHR",
    "Outpatient header - 1st position": "Outpatient",
    "Outpatient header - 2nd position": "Outpatient",
    "Claim": "Claims",
    "Inpatient claim": "Inpatient",
    "Inpatient header - 1st position": "Inpatient",
    "Inpatient header - 2nd position": "Inpatient",
    "Inpatient header - 3rd position": "Inpatient",
    "Inpatient header - 4th position": "Inpatient",
    "Inpatient header - 5th position": "Inpatient",
    "Inpatient header - 6th position": "Inpatient",
    "Inpatient header - 8th position": "Inpatient",
    "Inpatient header - 11th position": "Inpatient",
    "Inpatient header - 13th position": "Inpatient",
    "Inpatient header - 15th position": "Inpatient",
    "Admission diagnosis": "Inpatient",
    "Discharge diagnosis": "Inpatient",
    "Condition tested for by diagnostic procedure": "Secondary"
}
dataset_observation_df["observation_type_concept_name"] = dataset_observation_df["observation_type_concept_name"].map(mapping)


In [ ]:
dataset_observation_df['EHR'] = np.where((dataset_observation_df['observation_type_concept_name'] == 'EHR'), 1, 0)
dataset_observation_df['Claims'] = np.where((dataset_observation_df['observation_type_concept_name'] == 'Claims') | (dataset_observation_df['qualifier_concept_name'] == 'Billing'), 1, 0)
dataset_observation_df['Primary'] = np.where((dataset_observation_df['observation_type_concept_name'] == 'Primary') | (dataset_observation_df['qualifier_concept_name'] == 'Primary'), 1, 0)
dataset_observation_df['Secondary'] = np.where((dataset_observation_df['observation_type_concept_name'] == 'Secondary') | (dataset_observation_df['qualifier_concept_name'] == 'Secondary'), 1, 0)
for i in ['EHR', 'Claims', 'Primary', 'Secondary']:
    print(i, dataset_observation_df[i].sum())
dataset_observation_df[dataset_observation_df['observation_type_concept_name'] == 'Inpatient']['inpatient'] = 1


In [ ]:
import pandas
import os

dataset_person_sql = """
    SELECT
        person.person_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        p_race_concept.concept_name as race,
        p_ethnicity_concept.concept_name as ethnicity,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (702953) 
                    AND is_standard = 0 )) criteria ) )"""

dataset_person_df = pandas.read_gbq(
    dataset_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_person_df.head(5)


In [ ]:
import pandas as pd
import numpy as np
event_df1 = dataset_observation_df[['person_id', 'observation_datetime', 'EHR', 'Claims', 'Primary', 'Secondary', 'inpatient']]
event_df2 = dataset_condition_df[['person_id', 'condition_start_datetime', 'EHR', 'Claims', 'Primary', 'Secondary', 'inpatient']]
event_df2.columns = event_df1.columns
event_df = pd.concat([event_df1, event_df2], ignore_index=True)
aggregated_df = event_df.groupby('person_id', as_index=False).agg({
    'observation_datetime': 'min',  # earliest observation_datetime
    'EHR': 'max',              # largest inpatient flag
    'Claims': 'max',              # largest inpatient flag
    'Primary': 'max',              # largest inpatient flag
    'Secondary': 'max',              # largest inpatient flag
    'inpatient': 'max'
})

In [ ]:
covid_details_df = pd.merge(dataset_person_df, aggregated_df, on='person_id', how='inner')
covid_details_df['observation_datetime'] = pd.to_datetime(covid_details_df['observation_datetime'])
covid_details_df['date_of_birth'] = pd.to_datetime(covid_details_df['date_of_birth'])
covid_details_df['age'] = covid_details_df.apply(lambda row: (row['observation_datetime'].year - row['date_of_birth'].year) - ((row['observation_datetime'].month, row['observation_datetime'].day) < (row['date_of_birth'].month, row['date_of_birth'].day)), axis=1)

In [ ]:
# This query represents dataset "Covid and Comorbidities" for domain "condition" and was generated for All of Us Registered Tier Dataset v7
dataset_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (1326492, 1326493, 1326601, 1326602, 1326603, 1326604, 1326605, 1326606, 1326607, 1326608, 1326609, 1567956, 1567958, 1567959, 1567960, 1567964, 1567965, 1567966, 1567969, 1567971, 1569133, 1569134, 1569135, 1569145, 1569178, 1569179, 1569180, 1569487, 1569488, 1569489, 1569490, 1569491, 35206881, 35206882, 35207668, 35207702, 35207703, 35207704, 35207705, 35207706, 35207792, 35207793, 35208023, 35208024, 35208025, 37200198, 37200199, 37200200, 37200201, 37200202, 37200203, 37200204, 37200205, 37200206, 37200207, 37200208, 37200209, 37200210, 37200211, 37200212, 37200213, 37200214, 37200215, 37200216, 37200217, 37200218, 37200219, 37200220, 37200221, 37200222, 37200223, 37200224, 37200225, 37200227, 37200228, 37200229, 37200230, 37200232, 37200233, 37200234, 37200235, 37200237, 37200238, 37200239, 37200240, 37200242, 37200243, 37200244, 37200245, 37200246, 37200247, 37200248, 37200249, 37200251, 37200252, 37200253, 37200254, 45533019, 45533020,
 45533021, 45533022, 45533023, 45533439, 45533456, 45533457, 45537961, 45537962, 45538373, 45538374, 45538375, 45538377, 45542738, 45543167, 45543168, 45543182, 45543269, 45543270, 45547625, 45547626, 45547627, 45548010, 45548011, 45548012, 45548013, 45548022, 45548116, 45548117, 45552385, 45552386, 45557112, 45557113, 45557538, 45557539, 45557626, 45561949, 45562343, 45562344, 45562355, 45562457, 45566731, 45567167, 45567168, 45567180, 45567181, 45567265, 45567266, 45572168, 45572169, 45572170, 45572171, 45576443, 45576866, 45576868, 45576878, 45576951, 45576952, 45581352, 45581353, 45581354, 45581355, 45581766, 45581860, 45586139, 45586140, 45586572, 45586573, 45586574, 45586575, 45586587, 45586588, 45586674, 45586675, 45591027, 45591029, 45591030, 45591031, 45591456, 45591458, 45591459, 45591460, 45591461, 45591469, 45591558, 45591559, 45595797, 45595798, 45595799, 45596197, 45596198, 45596199, 45600641, 45600642, 45601024, 45601026, 45601027, 45601028, 45601038, 45601133, 45601134,
 45605401, 45605402, 45605403, 45605404, 45605405, 45605784, 45605785, 45605786, 45605787, 45605788, 1572195, 1568062, 1571486, 1567893, 1572190, 1569515, 1572193)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (702953) 
                            AND is_standard = 0 )) criteria ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_condition_df = pandas.read_gbq(
    dataset_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

In [ ]:
dataset_condition_df['source_concept_code'] = dataset_condition_df['source_concept_code'].str[:3]
full_data_df = pd.merge(covid_details_df['person_id'], dataset_condition_df[['person_id', 'source_concept_code']], on='person_id', how='left')
comorbidities_pivot = full_data_df.pivot_table(index='person_id', columns='source_concept_code', aggfunc='size', fill_value=0)
comorbidities_pivot = (comorbidities_pivot > 0).astype(int)
final_df = pd.merge(covid_details_df, comorbidities_pivot, on='person_id')

In [ ]:
final_df.columns

In [ ]:
final_df[['person_id', 'age', 'gender', 'race', 'ethnicity', 'observation_datetime', 'EHR', 'Claims', 'Primary',
       'Secondary', 'E03', 'E11', 'E78', 'I10', 'I25', 'I50', 'J44', 'J45', 'J96', 'N18', 'R06', 'R07', 'R09', 'inpatient']].to_csv('Covid and Comorbidities.csv')